In [1]:
!pip -q install yfinance pandas snowflake-connector-python snowflake-connector-python[pandas] statsmodels python-dotenv


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 646.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 97.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.4/479.4 kB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 8.3 MB/s eta 0:00:00


In [2]:
from datetime import datetime, timedelta
SYMBOLS = ["AAPL", "GOOGL"]
LOOKBACK_DAYS = 180
END_DATE = datetime.utcnow().date()
START_DATE = END_DATE - timedelta(days=LOOKBACK_DAYS)
print("Range:", START_DATE, "→", END_DATE, "Symbols:", SYMBOLS)


Range: 2025-04-09 → 2025-10-06 Symbols: ['AAPL', 'GOOGL']


/tmp/ipython-input-867183102.py:4: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  END_DATE = datetime.utcnow().date()


In [3]:
import yfinance as yf, pandas as pd
frames = []
for s in SYMBOLS:
    t = yf.Ticker(s)
    df = t.history(start=START_DATE, end=END_DATE, interval="1d", auto_adjust=False)
    if df.empty: print("WARNING: no data for", s); continue
    df = df.reset_index().rename(columns={"Date":"DATE","Open":"OPEN","High":"HIGH","Low":"LOW","Close":"CLOSE","Volume":"VOLUME"})
    df["SYMBOL"] = s
    frames.append(df[["SYMBOL","DATE","OPEN","HIGH","LOW","CLOSE","VOLUME"]])
if not frames: raise SystemExit("No data returned.")
prices_df = pd.concat(frames, ignore_index=True)
prices_df.head()
prices_df.tail()



,SYMBOL,DATE,OPEN,HIGH,LOW,CLOSE,VOLUME
241,GOOGL,2025-09-29 00:00:00-04:00,247.850006,251.149994,242.770004,244.050003,32505800
242,GOOGL,2025-09-30 00:00:00-04:00,242.809998,243.289993,239.250000,243.100006,34724300
243,GOOGL,2025-10-01 00:00:00-04:00,240.750000,246.300003,238.610001,244.899994,31658200
244,GOOGL,2025-10-02 00:00:00-04:00,245.149994,246.809998,242.300003,245.690002,25483300
245,GOOGL,2025-10-03 00:00:00-04:00,244.490005,246.300003,241.660004,245.350006,30232900


In [4]:
csv_path = "/content/stock_data.csv"
prices_df.to_csv(csv_path, index=False)
csv_path


'/content/stock_data.csv'

In [5]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [6]:
drive_csv_path = "/content/drive/MyDrive/stock_data.csv"
prices_df.to_csv(drive_csv_path, index=False)
drive_csv_path


'/content/drive/MyDrive/stock_data.csv'

In [7]:
import pandas as pd
drive_csv_path = "/content/drive/MyDrive/stock_data.csv"
prices_df = pd.read_csv(drive_csv_path, parse_dates=["DATE"])
prices_df.head()


,SYMBOL,DATE,OPEN,HIGH,LOW,CLOSE,VOLUME
0,AAPL,2025-04-09 00:00:00-04:00,171.949997,200.610001,171.889999,198.850006,184395900
1,AAPL,2025-04-10 00:00:00-04:00,189.070007,194.779999,183.000000,190.419998,121880000
2,AAPL,2025-04-11 00:00:00-04:00,186.100006,199.539993,186.059998,198.149994,87435900
3,AAPL,2025-04-14 00:00:00-04:00,211.440002,212.940002,201.160004,202.520004,101352900
4,AAPL,2025-04-15 00:00:00-04:00,201.860001,203.509995,199.800003,202.139999,51343900


In [8]:
import os

print("Available environment variables containing 'snowflake':")
for k in os.environ.keys():
    if "snowflake" in k.lower():
        print(" -", k)


Available environment variables containing 'snowflake':


In [9]:
!pip -q install cryptography snowflake-connector-python

from cryptography.hazmat.primitives import serialization
import snowflake.connector

SF_ACCOUNT = input("Snowflake ACCOUNT  ").strip()
SF_USER    = input("Snowflake USERNAME: ").strip()

private_key_path = "/content/drive/MyDrive/rsa_key.p8"


with open(private_key_path, "rb") as f:
    pkey = serialization.load_pem_private_key(f.read(), password=None)

pk_der = pkey.private_bytes(
    encoding=serialization.Encoding.DER,
    format=serialization.PrivateFormat.PKCS8,
    encryption_algorithm=serialization.NoEncryption()
)

# connect (defaults for db/schema can be set later with USE commands)
conn = snowflake.connector.connect(
    user=SF_USER,
    account=SF_ACCOUNT,
    private_key=pk_der,
    client_session_keep_alive=True
)

print(" Connected to Snowflake (key-pair authentication)")

Snowflake ACCOUNT  LVB17920
Snowflake USERNAME: JELLYFISH
 Connected to Snowflake (key-pair authentication)


In [10]:
with conn.cursor() as cur:
    cur.execute("USE DATABASE USER_DB_JELLYFISH;")
    cur.execute("USE SCHEMA RAW;")
    cur.execute("USE WAREHOUSE JELLYFISH_QUERY_WH;")
    print(cur.execute("SELECT CURRENT_DATABASE(), CURRENT_SCHEMA(), CURRENT_WAREHOUSE();").fetchone())


('USER_DB_JELLYFISH', 'RAW', 'JELLYFISH_QUERY_WH')


In [11]:

drive_csv_path = "/content/drive/MyDrive/stock_data.csv"
prices_df.to_csv(drive_csv_path, index=False)
print("Saved:", drive_csv_path)


prices_df = pd.read_csv(drive_csv_path, parse_dates=["DATE"])
prices_df.head()


Saved: /content/drive/MyDrive/stock_data.csv


,SYMBOL,DATE,OPEN,HIGH,LOW,CLOSE,VOLUME
0,AAPL,2025-04-09 00:00:00-04:00,171.949997,200.610001,171.889999,198.850006,184395900
1,AAPL,2025-04-10 00:00:00-04:00,189.070007,194.779999,183.000000,190.419998,121880000
2,AAPL,2025-04-11 00:00:00-04:00,186.100006,199.539993,186.059998,198.149994,87435900
3,AAPL,2025-04-14 00:00:00-04:00,211.440002,212.940002,201.160004,202.520004,101352900
4,AAPL,2025-04-15 00:00:00-04:00,201.860001,203.509995,199.800003,202.139999,51343900


In [12]:
# Rebuild prices_df from yfinance, then save to Drive

!pip -q install yfinance pandas

from datetime import datetime, timedelta
import yfinance as yf
import pandas as pd

# --- config ---
SYMBOLS = ["AAPL", "GOOGL"]
LOOKBACK_DAYS = 180
END_DATE = datetime.utcnow().date()
START_DATE = END_DATE - timedelta(days=LOOKBACK_DAYS)

print(f"Range: {START_DATE} → {END_DATE} | Symbols: {SYMBOLS}")

# --- fetch ---
frames = []
for s in SYMBOLS:
    t = yf.Ticker(s)
    df = t.history(start=START_DATE, end=END_DATE, interval="1d", auto_adjust=False)
    if df.empty:
        print(f" No data for {s}");
        continue
    df = (df.reset_index()
            .rename(columns={
                "Date":"DATE","Open":"OPEN","High":"HIGH",
                "Low":"LOW","Close":"CLOSE","Volume":"VOLUME"
            }))
    df["SYMBOL"] = s
    frames.append(df[["SYMBOL","DATE","OPEN","HIGH","LOW","CLOSE","VOLUME"]])

if not frames:
    raise SystemExit("No data returned from yfinance.")

prices_df = pd.concat(frames, ignore_index=True)

# --- clean types ---
prices_df["DATE"] = pd.to_datetime(prices_df["DATE"], errors="coerce").dt.tz_localize(None).dt.date
for c in ["OPEN","HIGH","LOW","CLOSE","VOLUME"]:
    prices_df[c] = pd.to_numeric(prices_df[c], errors="coerce")

print(prices_df.head())
print(prices_df.groupby("SYMBOL")["DATE"].agg(["min","max","count"]))

# --- save to Drive ---
drive_csv_path = "/content/drive/MyDrive/stock_data.csv"
prices_df.to_csv(drive_csv_path, index=False)
print("Saved:", drive_csv_path)


Range: 2025-04-09 → 2025-10-06 | Symbols: ['AAPL', 'GOOGL']
  SYMBOL        DATE        OPEN        HIGH         LOW       CLOSE  \
0   AAPL  2025-04-09  171.949997  200.610001  171.889999  198.850006   
1   AAPL  2025-04-10  189.070007  194.779999  183.000000  190.419998   
2   AAPL  2025-04-11  186.100006  199.539993  186.059998  198.149994   
3   AAPL  2025-04-14  211.440002  212.940002  201.160004  202.520004   
4   AAPL  2025-04-15  201.860001  203.509995  199.800003  202.139999   

      VOLUME  
0  184395900  
1  121880000  
2   87435900  
3  101352900  
4   51343900  
               min         max  count
SYMBOL                               
AAPL    2025-04-09  2025-10-03    123
GOOGL   2025-04-09  2025-10-03    123
Saved: /content/drive/MyDrive/stock_data.csv


/tmp/ipython-input-972808039.py:12: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  END_DATE = datetime.utcnow().date()


In [13]:
# Make sure your prices_df exists; if not, rebuild or load it first.
prices_df = prices_df.rename(columns={"DATE": "TRADE_DATE"})
prices_df["TRADE_DATE"] = pd.to_datetime(prices_df["TRADE_DATE"], errors="coerce").dt.tz_localize(None).dt.date
for c in ["OPEN","HIGH","LOW","CLOSE","VOLUME"]:
    prices_df[c] = pd.to_numeric(prices_df[c], errors="coerce")
prices_df.head()


,SYMBOL,TRADE_DATE,OPEN,HIGH,LOW,CLOSE,VOLUME
0,AAPL,2025-04-09,171.949997,200.610001,171.889999,198.850006,184395900
1,AAPL,2025-04-10,189.070007,194.779999,183.000000,190.419998,121880000
2,AAPL,2025-04-11,186.100006,199.539993,186.059998,198.149994,87435900
3,AAPL,2025-04-14,211.440002,212.940002,201.160004,202.520004,101352900
4,AAPL,2025-04-15,201.860001,203.509995,199.800003,202.139999,51343900


In [14]:
from snowflake.connector.pandas_tools import write_pandas
import traceback

DDL = """
CREATE TABLE IF NOT EXISTS RAW_PRICES (
  SYMBOL STRING NOT NULL,
  TRADE_DATE DATE NOT NULL,
  OPEN   NUMBER(18,6),
  HIGH   NUMBER(18,6),
  LOW    NUMBER(18,6),
  CLOSE  NUMBER(18,6),
  VOLUME NUMBER(38,0),
  LOAD_TS TIMESTAMP_NTZ DEFAULT CURRENT_TIMESTAMP(),
  CONSTRAINT PK_RAW PRIMARY KEY (SYMBOL, TRADE_DATE)
);
"""

with conn.cursor() as cur:
    try:
        # Make sure context is set
        cur.execute("USE DATABASE USER_DB_JELLYFISH")
        cur.execute("USE SCHEMA RAW")
        cur.execute("USE WAREHOUSE JELLYFISH_QUERY_WH")

        # (Re)create table if missing
        cur.execute("BEGIN")
        cur.execute(DDL)

        # Load DataFrame (ensure it has TRADE_DATE, not DATE)
        success, nchunks, nrows, _ = write_pandas(
            conn,
            prices_df,
            table_name="RAW_PRICES",
            database="USER_DB_JELLYFISH",
            schema="RAW",
            quote_identifiers=False
        )
        print(f"write_pandas success={success}, rows={nrows}, chunks={nchunks}")



        cur.execute("CREATE OR REPLACE TEMP TABLE TMP AS SELECT * FROM RAW_PRICES")

        cur.execute("TRUNCATE TABLE RAW_PRICES")
        cur.execute("""
            INSERT INTO RAW_PRICES (SYMBOL, TRADE_DATE, OPEN, HIGH, LOW, CLOSE, VOLUME, LOAD_TS)
            SELECT SYMBOL, TRADE_DATE, OPEN, HIGH, LOW, CLOSE, VOLUME, LOAD_TS
            FROM TMP
            QUALIFY ROW_NUMBER() OVER (
                PARTITION BY SYMBOL, TRADE_DATE
                ORDER BY LOAD_TS DESC
            ) = 1
        """)

        cur.execute("COMMIT")
        print(" Load committed and de-duplicated.")

    except Exception as e:
        cur.execute("ROLLBACK")
        print("Load failed; rolled back.")
        traceback.print_exc()
        raise


write_pandas success=True, rows=246, chunks=1
 Load committed and de-duplicated.


In [15]:
with conn.cursor() as cur:
    cur.execute("""
      SELECT SYMBOL,
             COUNT(*) AS ROW_COUNT,
             MIN(TRADE_DATE) AS MIN_DATE,
             MAX(TRADE_DATE) AS MAX_DATE
      FROM RAW_PRICES
      GROUP BY SYMBOL
      ORDER BY SYMBOL
    """)
    for row in cur.fetchall():
        print(row)


('AAPL', 180, datetime.date(2025, 1, 16), datetime.date(2025, 10, 3))
('GOOGL', 180, datetime.date(2025, 1, 16), datetime.date(2025, 10, 3))


In [16]:
with conn.cursor() as cur:
    # View with the columns Snowflake ML expects
    cur.execute("""
      CREATE OR REPLACE VIEW TRAIN_VIEW AS
      SELECT TRADE_DATE AS DATE, CLOSE, SYMBOL
      FROM RAW_PRICES
    """)

    # Forecast object (auto-ML)
    cur.execute("""
      CREATE OR REPLACE SNOWFLAKE.ML.FORECAST PREDICT_STOCK_PRICE (
        INPUT_DATA        => SYSTEM$REFERENCE('VIEW','TRAIN_VIEW'),
        SERIES_COLNAME    => 'SYMBOL',
        TIMESTAMP_COLNAME => 'DATE',
        TARGET_COLNAME    => 'CLOSE',
        CONFIG_OBJECT     => {'ON_ERROR':'SKIP'}
      )
    """)
print("Training view + ML forecast created.")


Training view + ML forecast created.


In [17]:
with conn.cursor() as cur:
    # Run forecast (90% prediction interval)
    cur.execute("""
      CALL PREDICT_STOCK_PRICE!FORECAST(
        FORECASTING_PERIODS => 7,
        CONFIG_OBJECT => {'prediction_interval': 0.90}
      )
    """)
    qid = cur.sfqid  # ID of the CALL above

    # Materialize results
    cur.execute(f"""
      CREATE OR REPLACE TABLE FORECAST_7D AS
      SELECT
        REPLACE(series,'"','') AS SYMBOL,
        ts::DATE               AS FORECAST_DATE,
        forecast               AS YHAT,
        lower_bound            AS YHAT_LOWER,
        upper_bound            AS YHAT_UPPER,
        CURRENT_TIMESTAMP()    AS CREATED_AT
      FROM TABLE(RESULT_SCAN('{qid}'))
    """)
print(" FORECAST_7D created.")


 FORECAST_7D created.


In [18]:
with conn.cursor() as cur:
    cur.execute("""
      CREATE OR REPLACE TABLE FINAL_PRICES AS
      SELECT
        SYMBOL,
        TRADE_DATE       AS PRICE_DATE,
        CLOSE            AS PRICE,
        'ACTUAL'         AS SOURCE
      FROM RAW_PRICES
      UNION ALL
      SELECT
        f.SYMBOL,
        f.FORECAST_DATE  AS PRICE_DATE,
        f.YHAT           AS PRICE,
        'FORECAST'       AS SOURCE
      FROM FORECAST_7D f
      LEFT JOIN RAW_PRICES r
        ON r.SYMBOL = f.SYMBOL AND r.TRADE_DATE = f.FORECAST_DATE
      WHERE r.SYMBOL IS NULL
    """)
print(" FINAL_PRICES created.")


 FINAL_PRICES created.


In [19]:
# counts
with conn.cursor() as cur:
    cur.execute("""
      SELECT SYMBOL, COUNT(*) AS ROW_COUNT,
             MIN(PRICE_DATE) AS MIN_DATE, MAX(PRICE_DATE) AS MAX_DATE
      FROM FINAL_PRICES
      GROUP BY SYMBOL ORDER BY SYMBOL
    """); print(cur.fetchall())

# latest 20 rows for screenshots
import pandas as pd
with conn.cursor() as cur:
    cur.execute("""
      SELECT * FROM FINAL_PRICES
      ORDER BY SYMBOL, PRICE_DATE DESC
      LIMIT 20
    """)
    r = cur.fetchall(); c = [d[0] for d in cur.description]
pd.DataFrame(r, columns=c)


[('AAPL', 187, datetime.date(2025, 1, 16), datetime.date(2025, 10, 14)), ('GOOGL', 187, datetime.date(2025, 1, 16), datetime.date(2025, 10, 14))]


,SYMBOL,PRICE_DATE,PRICE,SOURCE
0,AAPL,2025-10-14,255.721818,FORECAST
1,AAPL,2025-10-13,256.284582,FORECAST
2,AAPL,2025-10-10,255.941960,FORECAST
3,AAPL,2025-10-09,255.631681,FORECAST
4,AAPL,2025-10-08,256.005728,FORECAST
5,AAPL,2025-10-07,257.104862,FORECAST
6,AAPL,2025-10-06,257.408743,FORECAST
7,AAPL,2025-10-03,258.019989,ACTUAL
8,AAPL,2025-10-02,257.130005,ACTUAL
9,AAPL,2025-10-01,255.449997,ACTUAL


In [20]:
from google.colab import drive; drive.mount('/content/drive')
with conn.cursor() as cur:
    cur.execute("SELECT * FROM FINAL_PRICES ORDER BY SYMBOL, PRICE_DATE")
    rows = cur.fetchall(); cols = [d[0] for d in cur.description]
pd.DataFrame(rows, columns=cols).to_csv("/content/drive/MyDrive/final_prices.csv", index=False)
print("Saved /content/drive/MyDrive/final_prices.csv")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Saved /content/drive/MyDrive/final_prices.csv
